## <strong>環境構築</strong>

In [30]:
!pip install beautifulsoup4
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# スクレイピング

In [49]:
import re

import requests
from bs4 import BeautifulSoup

def getUrls(number):

    attrs = {
        'title': '経過'
    }
    vgm_url = 'https://www.shugiin.go.jp/internet/itdb_gian.nsf/html/gian/kaiji'+ str(number) +'.htm'
    html_text = requests.get(vgm_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    tracks = soup.find_all('a', attrs=attrs, string=re.compile(r'^((?!\().)*$'))

    for track in tracks:
        urlList.append('https://www.shugiin.go.jp/internet/itdb_gian.nsf/html/gian' + track.get('href')[1:])

urlList=[]
dietList = list(range(142, 210))
#有効な各議案のURLの一覧を取得します
#引数:国会の会期

for diet in dietList:
  getUrls(diet)

In [50]:
import re

import requests
from bs4 import BeautifulSoup


resultDictList = []




def createResultDict(url):

    resultDict = {'nameList':'','gianName':'','flag':0}

    attrs = {
      'summary': 'この表は横2列、縦2行の表です。'
    }
    vgm_url = str(url)
    html_text = requests.get(vgm_url).text
    soup = BeautifulSoup(html_text, 'html.parser')

    #議案提出
    tracks = soup.find_all('table', attrs=attrs)
    if tracks:

          giinList = tracks[0].findChildren("span")[1]
          resultGiinList = giinList.get_text().replace(' ','').replace(';',',') 
          resultDict['nameList'] = resultGiinList


    
    attrs = {
      'headers': 'NAIYO'
    }

    gian = soup.find_all('td', attrs=attrs)

    if gian:
      resultDict['gianName'] = gian[3].get_text()
      resultDictList.append(resultDict)

for url in urlList:
  createResultDict(url)


# csv出力

In [55]:
import csv
labels = ['nameList', 'gianName', 'flag']
try:
    with open('csv_dct.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in resultDictList:
          if elem['nameList']:
            writer.writerow(elem)
except IOError:
    print("I/O error")

ValueError: ignored